In [ ]:
## step 1 -- YOUTUBE LINK
from audio_load_youtube import download_youtube_audio
from pathlib import Path

url = 'https://www.youtube.com/watch?v=XZJ3fdV2Tkc' # replace with the 
# correct youtube link and run the notebook.


In [ ]:
result = download_youtube_audio(url=url) # saves to YOUTUBE_DIR returns a meta
print(result.__dict__)
audio_file = Path(result.file_path).name
result.file_path

In [ ]:
from pathlib import Path
from concall_audio_to_text import create_text_from_audio

# file = 'Delhivery_Ltd_Q4_FY2023-24_Earnings_Conference_Call.mp4'
file = Path(result.file_path).name
base_prmpt = "CONFERENCE CALL OF A COMPANY QUARTERLY EARNINGS" + file
## ---- USES WHISPER CREDIT ------ ###  

# note: update the size reduction in logs from converting to ogg
output_file = create_text_from_audio(file_name=file,
                                     base_prompt=base_prmpt,
                                     youtube=True ) # YT TRUE TO make sure correct dir is used
    
output_file

In [ ]:
# create corpus from text for retrieval 
from concall_text_to_kowledge import create_text_meta_doc


# file = 'Delhivery_Ltd_Q4_FY2023-24_Earnings_Conference_Call.mp4'
file = Path(result.file_path).name
base_prmpt = "CONFERENCE CALL OF A COMPANY QUARTERLY EARNINGS" + file
summary_prmpt_concall = f"FILE: {file} ||  You are a helpful assistant to summarise a quarterly EARNINGS CONFERENCE CALL. The transcript of the call will be provided in chunks as context and you have to extract information carefully and concisely in summaries.  "
sections_concall =  ['INTRO', 'MANAGEMENT NOTE', 'ANALYST QA', 'CONCLUSION']

chunk_size= 4000

summary_file = create_text_meta_doc(ts_filename=file,
                                    chunk_size=chunk_size,
                                    summariser_prompt=summary_prmpt_concall,
                                    sections=sections_concall)
summary_file

In [ ]:
## step 4 
from concall_text_to_kowledge import create_embeddings_from_chunk_doc


embedding_doc_file = create_embeddings_from_chunk_doc(filename=file)
embedding_doc_file

## Corpus generated, now ask questions 

In [ ]:
# step 5 - final step
 
from concall_question_over_knowledge import answer_question, get_context_corpus

organisation = 'Delhivery'
filename =  'Delhivery_Ltd_Q4_FY2023-24_Earnings_Conference_Call.json'
question_prompt = f" You will be provided with transcript chunks of a conference call. \
                    Followed by a question from an investor who is either invested in the  company or is looking to invest for mid to long term. Answer such that \
                    you are expert stocks analyst. All currency are in INDIAN RUPEES (INR). \
                    Answer without any starting phrases like '..here are the' etc. Only reply with the required information in concise but detailed manner \
                     \n  ORGANISATION : {organisation} \n FILENAME : {filename}"

def test_answer_from_gpt_delhivery( question_prompt=question_prompt):
    doc = get_context_corpus(file_name=filename,)
    
    # question = "what is this call about?"
    # question = "What was the overall performance of the company in the last quarter?"
    question = "was there a huge drop in profits, revenue or margins this quarter? Should I be worried about anything  "
    question = "what are some signs I could be on the lookout for to see if performance of company is improving in the coming months in accordance with the management's goals and commitments mentioned in this call?"
    question = "what are the financial numbers I can expect in next quarter based on this?"
    question = "Key concerns raised by the analysts in QA?"
    # question = "What will be their approximate revenue in coming two quarters comparing with YoY and this Quarter."
    # question = "what measures the management is taking to keep the revenue and margins up in coming quarters?"
    # question = "how does the management keep their employees motivated or follow work life balance? I want to understand the work culture in Morepen Lab."
    # question = "what are the main products and revenue sources of this company?"

    # question = "what were the key areas of performance growth in this quarter according to the management?"
    # question = "Summarise the analyst QnA of the call in around 600 words?"
    # # question = "when is next tom cruise movie coming?"
    # question = "what are the revenues and profit numbers of their competitors this quarter? If not available then just give me the competitor companies."
    # question = "what does this company do? "
    # question = "summarise the management's take on the company's performance along with analysts's concerns and key take aways."
    
    return  answer_question(doc,question,
                            file_name=filename,
                            question_prompt=question_prompt,
                            _top_n=3)
test_answer_from_gpt_delhivery()